# Testing of Spell Framework

In [1]:
import pandas as pd

import Reader
import LogKeysManager
import ParamsExtractor
import DataPreprocessor
import ReduceDim
import ClusterData
import Spell
import Spell2

2024-06-04 13:20:58.682070: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 13:20:58.682703: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 13:20:58.686772: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 13:20:58.730456: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 13:20:59.569139: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
# Testing Params Extractor
#log_types = ['cron', 'user']
log_types = ['cron', 'laurel', 'maillog', 'messages', 'secure', 'user']
file_paths = ['../../../temp_logs/login01.' + logtype + '.log' for logtype in log_types]

df_list = []

for file_path in file_paths:
    r = Reader.Reader(file_path)
    df = r.read_file()
    df_list.append(df)

In [3]:
print(df_list[0].head())

      host  ident    pid                                            message  \
0  login01  CROND  53766  (root) CMD (/cinecalocal/scripts/clean_shm_fil...   
1  login01  CROND  53767  (root) CMD ( /cinecalocal/nagios/passive/passi...   
2  login01  CROND  53822  (a07cmc01) CMD (source /g100_work/CMCC_medfs_0...   
3  login01  CROND  53823  (a07cmc01) CMD (/g100_work/CMCC_medfs_0/downlo...   
4  login01  CROND  55382            (root) CMD (run-parts /etc/cron.hourly)   

  severity facility                       time  
0     info     cron  2024-03-27 00:00:01 +0100  
1     info     cron  2024-03-27 00:00:01 +0100  
2     info     cron  2024-03-27 00:00:01 +0100  
3     info     cron  2024-03-27 00:00:01 +0100  
4     info     cron  2024-03-27 00:01:01 +0100  


In [4]:
logs = [[m for m in df_list[i]['message']] for i in range(len(df_list))] # we exclude laurel now, te be able to get better logs and experiment

In [5]:
df_after = []

for i, df in enumerate(df_list):
    p = ParamsExtractor.ParamsExtractor(df)
    df = p.convert_params(df)
    new_df = p.get_params()
    df_after.append(new_df)


for i in range(len(df_list)):
    #print('=================')
    #print(df_list[i].head())
    #print(df_list[i].columns)
    #print(df_list[i].iloc[0])
    #print('********')
    #print(df_after[i].head())
    #print(df_after[i].columns)
    #print(df_after[i].iloc[0])
    # Concatenate df and df_after
    df_list[i] = pd.concat([df_list[i], df_after[i]], axis=1)
    #print(df_list[i].columns)

In [6]:
similarities = [0.4, 0.0, 0.3, 0.45, 0.45, 0.5]
for i,log_t in enumerate(logs):
    print('============================================')
    if i == 1:
        print('*')
    else:
        spell = Spell.Spell(similarity=similarities[i])
        spell.parse_logs(log_t)
        spell.display_templates()

Template: (root) CMD (/cinecalocal/scripts/clean_shm_files.sh)
Template: (root) CMD * * * * * * * *
Template: (a07cmc01) CMD (source /g100_work/CMCC_medfs_0/download/set_env.sh; python /g100_work/CMCC_medfs_0/download/dasDwlSystem/mp_dwld_driver.py > /g100_work/CMCC_medfs_0/download/LOGS/dasDwlSystem/mp_dwld_driver_log_`date +%Y%m%d%H%M%S`.log 2>&1)
Template: (a07cmc01) CMD * * * 2>&1)
Template: (root) CMD * *
Template: (/etc/cron.hourly) starting 0anacron
Template: Anacron started on 2024-03-27
Template: Normal exit (0 jobs run)
Template: (/etc/cron.hourly) * 0anacron
Template: (root) CMD * * * * * *
Template: (acctdata) CMD ( /bin/timeout 239m /cineca/asdata/bin/cindata.pl g >/tmp/lastcron_cindata 2>&1)
Template: (root) CMD * * *
Template: (a07ccc02) CMD (/g100/home/usera07ccc/a07ccc02/acct_data/scripts_in_cron/sync_acct-data_to_dres.sh)
Template: (a07ccc02) CMD *
Template: (a07cmc01) CMD (source /g100_work/CMCC_medfs_0/download/set_env.sh; python * * * * * * * * > * +%Y%m%d%H%M%S`.l

In [7]:
file_path = 'log_key.json'
l = LogKeysManager.LogKeysManager(file_path)
log_keys = l.get_log_keys()


for k in log_keys:
    print(f"Log Key :{k}")
    print(log_keys[k])

Log Key :*
0
Log Key : * --delay 20 --cronid 1 1>* 2>&1
1
Log Key :source *; python * > * +%Y%m%d%H%M%S`.log 2>&1
2
Log Key :* >> * 2>&1
3
Log Key :run-parts *
4
Log Key :(*) starting 0anacron
5
Log Key :Anacron started on 2024-03-27
6
Log Key :Normal exit (*)
7
Log Key :(*) finished 0anacron
8
Log Key :* "STATUS_CHECK" -p local2.info >> *
9
Log Key : * --delay 20 --cronid 2 1>* 2>&1
10
Log Key : * 239m * g  >*  2>&1
11
Log Key :* 1>* 2>*
12
Log Key :source *; python * -p $(date +%Y%m%d
13
Log Key :Will run job `cron.daily' in 28 min.
14
Log Key :Jobs will be executed sequentially
15
Log Key :Job `cron.daily' started
16
Log Key :(*) starting logrotate
17
Log Key :(*) finished logrotate
18
Log Key :Job `cron.daily' terminated
19
Log Key :[ ! -s * ] && * 's*/' * > *
20
Log Key :* -m rpm -a >* 2>&1
21
Log Key :sh * >> * 2>&1
22
Log Key :source *; python * -p $(date -d "yesterday" +%Y%m%d
23
Log Key :. *; . *; module load python*; ${HOME}* --cluster ${HOME}* --path_statistics ${HOME}* --pa